# Analisando dados de espécies em perigo de extinção

### Este caderno é um experimento para importar e tratar dados armazenados em uma planilha do Microsoft Excel que contém informações sobre as espécies em risco de extinção nas principiais áreas de preservação e parques nacionais brasileiros.

Os dados utilizados neste caderno foram obtidos no [Portal Brasileiro de Dados Abertos](https://dados.gov.br/). 
A autoria dos dados é da CNCFlora.

#### Bibliografia:

CNCFlora (2016). Espécies Ameaçadas da Flora em UCs Federais, versão 2016-10-03. Disponível em:http://ckan.jbrj.gov.br/dataset/ameacadas-em-ucs-federais. Acesso em 30/07/2021 (data de download)

#### Carregando dependencias, fonte de dados e dataframe do Pandas

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as db
import IPython

src = input("Insira o nome da planilha, ou tecle Enter para usar o nome padrão: ")
if len(src) < 1:
    src = "fauna_fed.xlsx"
data = pd.read_excel(src)
print("Dados carregados como: " + str(type(data)))


Insira o nome da planilha, ou tecle Enter para usar o nome padrão: 
Dados carregados como: <class 'pandas.core.frame.DataFrame'>


#### Função que permite visualizar dados rapidamente em uma tabela (primeiras 50 linhas)

In [2]:
def ldf_display(df, lines=50):
    txt = ("<iframe " +
           "srcdoc='" + df.head(lines).to_html() + "' " +
           "width=1000 height=300>" +
           "</iframe>")

    return IPython.display.HTML(txt)

ldf_display (data)

D:\Dev\lib\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


,Número identificador do táxon,Divisão em: Invertebrados ou Vertebrados,Número da Portaria vigente (Portarias MMA 444/2014 e 445/2014),"Divisão em: Anfíbios, Aves, Invertebrados Aquáticos, Invertebrados Terrestres, Mamíferos, Peixes Continentais, Peixes Marinhos ou Répteis",Classe segundo classificação taxonômica,Ordem segundo classificação taxonômica,Família segundo classificação taxonômica,Táxon ao qual as informações dos demais campos se refere. De acordo com nomes do Livro Vermelho 2018,Táxon ao qual as informações dos demais campos se refere. De acordo com as portarias MMA 444/2014 e 445/2014,Categoria do estado de conservação do táxon validada na oficina de validação,Unidade de conservação de ocorrência da espécie,Unnamed: 11,Código CNUC da UC de ocorrência da espécie,Estado sobre ocorrência em UC da espécie,Unnamed: 14,Unnamed: 15
0,8458,Vertebrados,444/2014,Anfíbios,Amphibia,Anura,Leptodactylidae,NaN,Physalaemus maximus,VU,APA de Araponga,MG,?,Presença em UC,NaN,Livro Vermelho 2018
1,11070,Vertebrados,444/2014,Mamíferos,Mammalia,Cingulata,Dasypodidae,NaN,Priodontes maximus,VU,PE Serra de Santa Bárbara,MT,0000.51.0462,Presença em UC,NaN,Livro Vermelho 2018
2,11132,Vertebrados,444/2014,Mamíferos,Mammalia,Perissodactyla,Tapiriidae,NaN,Tapirus terrestris,VU,PE Serra de Santa Bárbara,MT,0000.51.0462,Presença em UC,NaN,Livro Vermelho 2018
3,10802,Vertebrados,444/2014,Mamíferos,Mammalia,Artiodacyla,Tayassuidae,NaN,Tayassu pecari,VU,PE Serra de Santa Bárbara,MT,0000.51.0462,Presença em UC,NaN,Livro Vermelho 2018
4,2355,Invertebrados,445/2014,Invertebrados Aquáticos,Malacostraca,Decapoda,Aeglidae,NaN,Aegla brevipalma,CR,* Desconhecida,-,-,Desconhecido,NaN,Livro Vermelho 2018
5,2356,Invertebrados,445/2014,Invertebrados Aquáticos,Malacostraca,Decapoda,Aeglidae,NaN,Aegla camargoi,EN,* Desconhecida,-,-,Desconhecido,NaN,Livro Vermelho 2018
6,2359,Invertebrados,445/2014,Invertebrados Aquáticos,Malacostraca,Decapoda,Aeglidae,NaN,Aegla franca,CR,* Desconhecida,-,-,Desconhecido,NaN,Livro Vermelho 2018
7,2361,Invertebrados,445/2014,Invertebrados Aquáticos,Malacostraca,Decapoda,Aeglidae,NaN,Aegla grisella,VU,* Desconhecida,-,-,Desconhecido,NaN,Livro Vermelho 2018
8,2363,Invertebrados,445/2014,Invertebrados Aquáticos,Malacostraca,Decapoda,Aeglidae,NaN,Aegla inermis,EN,* Desconhecida,-,-,Desconhecido,NaN,Livro Vermelho 2018
9,2364,Invertebrados,445/2014,Invertebrados Aquáticos,Malacostraca,Decapoda,Aeglidae,NaN,Aegla itacolomiensis,EN,* Desconhecida,-,-,Desconhecido,NaN,Livro Vermelho 2018


#### Há muitas strings repetidas; vamos transformá-las em chaves primárias em tabelas específicas, para que possamos utilizá-las como chaves estrangeiras na tabela final:

In [3]:
def criar_CP (fonte, nome_coluna, lista): #onde fonte é um dataframe Pandas e lista é uma variável onde serão armazenados os dicionários criados
    kp = dict()
    chave = 0
    for item in fonte[nome_coluna]:
        if item in kp:
            continue
        kp[item] = chave
        chave += 1 
    lista.append(kp)
    print("Chaves criadas para elementos da coluna: " + nome_coluna)

tabelas = list()
for col in data.columns[3:7]:
    criar_CP(data, col, tabelas)
criar_CP(data, "Unidade de conservação de ocorrência da espécie", tabelas)
criar_CP(data, "Unnamed: 11", tabelas) #Tabela CP de unidades federais


Chaves criadas para elementos da coluna: Divisão em: Anfíbios, Aves, Invertebrados Aquáticos, Invertebrados Terrestres, Mamíferos, Peixes Continentais, Peixes Marinhos ou Répteis
Chaves criadas para elementos da coluna: Classe segundo classificação taxonômica
Chaves criadas para elementos da coluna: Ordem segundo classificação taxonômica
Chaves criadas para elementos da coluna: Família segundo classificação taxonômica
Chaves criadas para elementos da coluna: Unidade de conservação de ocorrência da espécie
Chaves criadas para elementos da coluna: Unnamed: 11


#### Criando base de dados SQLite e inserindo tabelas relacionais

In [4]:
db.register_adapter(np.int32, int)  #Necessário para que o SQLite aceite ints com mais de 8 bits, como é o caso das ints geradas pelo pandas
db.register_adapter(np.int64, int)

conn = db.connect("fauna_db.sqlite")
cur = conn.cursor()

cur.executescript("""
DROP TABLE IF EXISTS divisao;
DROP TABLE IF EXISTS classe;
DROP TABLE IF EXISTS ordem;
DROP TABLE IF EXISTS familia;
DROP TABLE IF EXISTS UC;
DROP TABLE IF EXISTS UF;
DROP TABLE IF EXISTS geral;

CREATE TABLE geral (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    especie TEXT ,
    divisao_id INTEGER,
    classe_id INTEGER,
    ordem_id INTEGER,
    familia_id INTEGER,
    UC_id INTEGER,
    UF_id INTEGER
);

CREATE TABLE divisao (
    id  INTEGER NOT NULL PRIMARY KEY UNIQUE,
    nome    TEXT UNIQUE
);

CREATE TABLE classe (
    id  INTEGER NOT NULL PRIMARY KEY UNIQUE,
    nome    TEXT UNIQUE
);

CREATE TABLE ordem (
    id  INTEGER NOT NULL PRIMARY KEY UNIQUE,
    nome    TEXT UNIQUE
);
CREATE TABLE familia (
    id  INTEGER NOT NULL PRIMARY KEY UNIQUE,
    nome    TEXT UNIQUE
);
CREATE TABLE UC (
    id  INTEGER NOT NULL PRIMARY KEY UNIQUE,
    nome    TEXT UNIQUE
);
CREATE TABLE UF (
    id  INTEGER NOT NULL PRIMARY KEY UNIQUE,
    nome    TEXT UNIQUE
);

                 """)

titulos = ["divisao", "classe", "ordem", "familia", "UC", "UF"]
i = 0
for tabela in tabelas:
    for nome in tabela:
        cur.execute("INSERT OR IGNORE INTO " +titulos[i] + " (id, nome) VALUES (?, ?)", (tabela[nome], nome, ))
    conn.commit()
    print("Valores inseridos na tabela \"" + titulos[i] + "\" do banco de dados")
    i += 1
    

Valores inseridos na tabela "divisao" do banco de dados
Valores inseridos na tabela "classe" do banco de dados
Valores inseridos na tabela "ordem" do banco de dados
Valores inseridos na tabela "familia" do banco de dados
Valores inseridos na tabela "UC" do banco de dados
Valores inseridos na tabela "UF" do banco de dados


#### Mapeando chaves estrangeiras no dataframe pandas

In [5]:
headers = list()
for col in data.columns[3:7]:
    headers.append(col)
headers.append("Unidade de conservação de ocorrência da espécie")
headers.append("Unnamed: 11")
for i in range (len(tabelas)):  
    print ("Mapendo chaves estrangeiras na coluna:", headers[i])
    data[headers[i]] = data[headers[i]].map(tabelas[i]) #substitui todos os valores por suas respectivas chaves na tabela com base no dicionário presente em tabelas

Mapendo chaves estrangeiras na coluna: Divisão em: Anfíbios, Aves, Invertebrados Aquáticos, Invertebrados Terrestres, Mamíferos, Peixes Continentais, Peixes Marinhos ou Répteis
Mapendo chaves estrangeiras na coluna: Classe segundo classificação taxonômica
Mapendo chaves estrangeiras na coluna: Ordem segundo classificação taxonômica
Mapendo chaves estrangeiras na coluna: Família segundo classificação taxonômica
Mapendo chaves estrangeiras na coluna: Unidade de conservação de ocorrência da espécie
Mapendo chaves estrangeiras na coluna: Unnamed: 11


In [6]:
ldf_display (data)

D:\Dev\lib\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


,Número identificador do táxon,Divisão em: Invertebrados ou Vertebrados,Número da Portaria vigente (Portarias MMA 444/2014 e 445/2014),"Divisão em: Anfíbios, Aves, Invertebrados Aquáticos, Invertebrados Terrestres, Mamíferos, Peixes Continentais, Peixes Marinhos ou Répteis",Classe segundo classificação taxonômica,Ordem segundo classificação taxonômica,Família segundo classificação taxonômica,Táxon ao qual as informações dos demais campos se refere. De acordo com nomes do Livro Vermelho 2018,Táxon ao qual as informações dos demais campos se refere. De acordo com as portarias MMA 444/2014 e 445/2014,Categoria do estado de conservação do táxon validada na oficina de validação,Unidade de conservação de ocorrência da espécie,Unnamed: 11,Código CNUC da UC de ocorrência da espécie,Estado sobre ocorrência em UC da espécie,Unnamed: 14,Unnamed: 15
0,8458,Vertebrados,444/2014,0,0,0,0,NaN,Physalaemus maximus,VU,0,0,?,Presença em UC,NaN,Livro Vermelho 2018
1,11070,Vertebrados,444/2014,1,1,1,1,NaN,Priodontes maximus,VU,1,1,0000.51.0462,Presença em UC,NaN,Livro Vermelho 2018
2,11132,Vertebrados,444/2014,1,1,2,2,NaN,Tapirus terrestris,VU,1,1,0000.51.0462,Presença em UC,NaN,Livro Vermelho 2018
3,10802,Vertebrados,444/2014,1,1,3,3,NaN,Tayassu pecari,VU,1,1,0000.51.0462,Presença em UC,NaN,Livro Vermelho 2018
4,2355,Invertebrados,445/2014,2,2,4,4,NaN,Aegla brevipalma,CR,2,2,-,Desconhecido,NaN,Livro Vermelho 2018
5,2356,Invertebrados,445/2014,2,2,4,4,NaN,Aegla camargoi,EN,2,2,-,Desconhecido,NaN,Livro Vermelho 2018
6,2359,Invertebrados,445/2014,2,2,4,4,NaN,Aegla franca,CR,2,2,-,Desconhecido,NaN,Livro Vermelho 2018
7,2361,Invertebrados,445/2014,2,2,4,4,NaN,Aegla grisella,VU,2,2,-,Desconhecido,NaN,Livro Vermelho 2018
8,2363,Invertebrados,445/2014,2,2,4,4,NaN,Aegla inermis,EN,2,2,-,Desconhecido,NaN,Livro Vermelho 2018
9,2364,Invertebrados,445/2014,2,2,4,4,NaN,Aegla itacolomiensis,EN,2,2,-,Desconhecido,NaN,Livro Vermelho 2018


#### Populando tabela principal (geral) da base de dados:

In [7]:
for i in range(len(data)):
    especie = data[data.columns[8]][i]
    divisao_id = data[data.columns[3]][i]
    classe_id = data[data.columns[4]][i]
    ordem_id = data[data.columns[5]][i]
    familia_id = data[data.columns[6]][i]
    UC_id = data[data.columns[10]][i]
    UF_id = data[data.columns[11]][i]
    cur.execute("""INSERT INTO geral (especie, divisao_id, classe_id, ordem_id, familia_id, UC_id, UF_id) VALUES (?, ?, ?, ?, ?, ?, ?)""", (especie, divisao_id, classe_id, ordem_id, familia_id, UC_id, UF_id, ))    
conn.commit()
print("Tabela geral populada")



Tabela geral populada
